# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering,DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import metrics
from sklearn.cluster import KMeans

import warnings


# Load the dataset

In [ ]:
df = pd.read_csv('CC GENERAL.csv')
df.head()

# EDA

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
# fill the missing values with means
df["MINIMUM_PAYMENTS"] = df['MINIMUM_PAYMENTS'].fillna(df["MINIMUM_PAYMENTS"].mean())
df["CREDIT_LIMIT"] = df['CREDIT_LIMIT'].fillna(df["CREDIT_LIMIT"].mean())

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop(columns="CUST_ID", inplace=True)

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(30,45))
for i, col in enumerate(df.columns):
    if df[col].dtype != "object":
        ax = plt.subplot(9,2,i+1)
        sns.kdeplot(df[col],ax=ax)
        plt.xlabel(col)
plt.show() 

In [ ]:
plt.figure(figsize=(10,60))
for i in range(0,17):
    plt.subplot(17,1,i+1)
    sns.distplot(df[df.columns[i]],kde_kws={'color':'b','bw': 0.1,'lw':3,'label':'KDE'},hist_kws={'color':'r'})
    plt.title(df.columns[i])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(), annot=True)
plt.show()

# Scaling the dataframe

In [ ]:
scalar = StandardScaler()
scaled_df = scalar.fit_transform(df)
scaled_df.shape

# Dimensionality reduction
converting to 2D df using PCA

In [ ]:
pca = PCA(n_components=2)
priciple_component = pca.fit_transform(scaled_df)
pca_df = pd.DataFrame(data=priciple_component, columns=['PCA1', 'PCA2'])
pca_df


# Find the K value using the elbow method

In [ ]:
type(scaled_df)

In [ ]:
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


# Scale the data using StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

# Determine the optimal number of clusters using the elbow method
range_val = range(1, 11)  # Range of cluster numbers you want to try
inertia = []

for i in range_val:
    kmean = KMeans(n_clusters=i)
    labels = kmean.fit_predict(scaled_df)  # Using fit_predict here
    inertia.append(kmean.inertia_)

# Plotting the elbow curve
plt.plot(range_val, inertia, 'bx-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal Clusters')
plt.show()
